In [4]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
#tf.test.is_gpu_available()
tf.config.list_physical_devices('GPU')

[]

In [6]:
batchsize=8

In [7]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'C:\Users\91900\Desktop\Eye_dataset\mrlEyes_2018_01\Prepared_Data\Train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'C:\Users\91900\Desktop\Eye_dataset\mrlEyes_2018_01\Prepared_Data\Train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 64470 images belonging to 2 classes.
Found 16116 images belonging to 2 classes.


In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\91900\Desktop\Eye_dataset\mrlEyes_2018_01\Prepared_Data\Test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 4312 images belonging to 2 classes.


In [9]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 39, 39, 32)   0           ['batch_normalization[0][0]']

In [12]:
model.layers

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [10]:
checkpoint = ModelCheckpoint(r'C:\Users\91900\Desktop\Eye_dataset\mrlEyes_2018_01\model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [11]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

Epoch 1/5
8058/8058 [==============================] - ETA: 0s - loss: 0.2043 - accuracy: 0.9198
Epoch 1: val_loss improved from inf to 0.19695, saving model to C:\Users\91900\Desktop\Eye_dataset\mrlEyes_2018_01\model.h5
8058/8058 [==============================] - 876s 108ms/step - loss: 0.2043 - accuracy: 0.9198 - val_loss: 0.1969 - val_accuracy: 0.9222 - lr: 0.0010
Epoch 2/5
8058/8058 [==============================] - ETA: 0s - loss: 0.1776 - accuracy: 0.9306
Epoch 2: val_loss did not improve from 0.19695
8058/8058 [==============================] - 901s 112ms/step - loss: 0.1776 - accuracy: 0.9306 - val_loss: 0.2036 - val_accuracy: 0.9147 - lr: 0.0010
Epoch 3/5
8058/8058 [==============================] - ETA: 0s - loss: 0.1709 - accuracy: 0.9353
Epoch 3: val_loss did not improve from 0.19695
8058/8058 [==============================] - 592s 74ms/step - loss: 0.1709 - accuracy: 0.9353 - val_loss: 0.2018 - val_accuracy: 0.9163 - lr: 0.0010
Epoch 4/5
8058/8058 [=====================